# Prevayl Technical Task

Task 1

- From the data available in the data/transformer folder provided. Can you please complete the following:
  
  a) construct a simple transformer model using a library of your choice. (If you are struggling to get started have a look at this implementation https://keras.io/examples/generative/text_generation_with_miniature_gpt/)
  
  b) Train the model (Ok to train this on a smaller sample if necessary)
  
  c) Test the model and report the model accuracy
  
  d) Provide and evaluation of the accuracy and what could be done to improve the accuracy and robustness of the model

Task 2

- From the data available in the data/health folder provided. Can you please complete the following:

  a) Perform a exploratory data analysis on the dataset as if you were preparing it to build a predictive model.
  
  b) Provide a summary of your findings


# Task 1

There are some guidelines below to how you may want to structure your work and download the data but feel free to structure your code as you wish. If you want to write out your work via scripts too then that is ok. 

## Load data


In [1]:
import json

In [4]:
%ls data

health/      transformer/


In [5]:
# Suggested way to filter the dataset. Feel free to remove this if you want to

# Load the full dataset
with open(r"data/transformer/winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)

# Filter the dataset
filtered_data = [
    "wine review : "
    + x["country"]
    + " : "
    + x["province"]
    + " : "
    + x["variety"]
    + " : "
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None]

#limit the number of examples we're using for training because my laptop is very slow
filtered_data = filtered_data

In [12]:
filtered_data

["wine review : Italy : Sicily & Sardinia : White Blend : Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
 "wine review : Portugal : Douro : Portuguese Red : This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
 'wine review : US : Oregon : Pinot Gris : Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'wine review : US : Michigan : Riesling : Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.',
 "wine r

## Tokenize the data



In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample data
data = [
    "wine review : Italy : Sicily & Sardinia : White Blend : Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
    "wine review : Portugal : Douro : Portuguese Red : This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016."
]

# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(data)

# Pad sequences
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Define vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token

# Prepare input and output data for training
X = padded_sequences[:, :-1]
y = padded_sequences[:, 1:]

# Convert labels to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

print("Vocabulary Size:", vocab_size)
print("Max Sequence Length:", max_sequence_length)
print("Input Shape:", X.shape)
print("Output Shape:", y.shape)


ModuleNotFoundError: No module named 'tensorflow'

## Create the training set

## Create the Transformer

## Build Model

## Train Model

## Generate text using transformer

## Summary


# Task 2

Again, there are some options of how you may want to lay out your work below but feel free to put it into a format that you think is best. The data in the health folder is split into raw and final. We suggest first looking at the raw data and then the final test and train in your exploration. If you want more context regarding the data there is information in the papers folder that tells you where this data is from.

In [ ]:
# imports

## Load Data

## Data Exploration


## Summary